# nb_01_ingestao_bronze
**Objetivo:** realizar a ingestão dos dados brutos extraídos do banco AdventureWorks2017 para a camada Bronze, preservando a estrutura original das tabelas e garantindo rastreabilidade dos dados.


In [0]:
%sql
-- Cria catálogo e schemas
CREATE CATALOG IF NOT EXISTS mvp_catalog;
USE CATALOG mvp_catalog;

CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;

-- Cria Volumes
CREATE VOLUME IF NOT EXISTS bronze.bronze_data;
CREATE VOLUME IF NOT EXISTS silver.silver_data;
CREATE VOLUME IF NOT EXISTS gold.gold_data;

SHOW CATALOGS;
SHOW SCHEMAS;



In [0]:
%python
BRONZE_VOLUME = "/Volumes/mvp_catalog/bronze/bronze_data"
DELTA_BASE = f"{BRONZE_VOLUME}/delta"

print("BRONZE_VOLUME =", BRONZE_VOLUME)
print("DELTA_BASE =", DELTA_BASE)


In [0]:
%sql
-- Listar conteúdo do volume para confirmar onde o CSV foi colocado
LIST '/Volumes/mvp_catalog/bronze/bronze_data/';

In [0]:
%python
tables = {
    "sales_order_header": "SalesOrderHeader.csv",
    "sales_order_detail": "SalesOrderDetail.csv",
    "product": "Product.csv",
    "product_subcategory": "ProductSubcategory.csv",
    "product_category": "ProductCategory.csv",
    "customer": "Customer.csv",
    "person": "Person.csv",
    "email_address": "EmailAddress.csv",
    "sales_territory": "SalesTerritory.csv",
    "country_region": "CountryRegion.csv"
}

for table, file in tables.items():
    print(f"{table} -> {BRONZE_VOLUME}/{file}")


In [0]:
%python
def ingest_csv_to_bronze(table_name, file_name):
    path = f"{BRONZE_VOLUME}/{file_name}"

    df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("sep", ";")
        .csv(path)
    )

    (
        df.write
          .mode("overwrite")
          .format("delta")
          .saveAsTable(f"bronze.{table_name}")
    )

    print(f"bronze.{table_name} criada com sucesso")



In [0]:
%python
for table_name, file_name in tables.items():
    ingest_csv_to_bronze(table_name, file_name)


In [0]:
%sql
SHOW TABLES IN bronze;


In [0]:
%sql
SELECT COUNT(*) FROM bronze.sales_order_header;


In [0]:
%sql
SELECT COUNT(*) FROM bronze.sales_order_detail;